In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

kaggle_input="./kaggle/input"
# kaggle_working="./kaggle/working"
kaggle_temp="./kaggle/temp"

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [25]:
f_test=pd.read_csv(f"{kaggle_temp}/f_test.csv")
f_test


,eeg_id,savg0,savg1,savg2,savg3,savg4,savg5,savg6,savg7,savg8,savg9,savg10,savg11,savg12,savg13,savg14,savg15,savg16,savg17,savg18,savg19,savg20,savg21,savg22,savg23,savg24,savg25,savg26,savg27,savg28,savg29,savg30,savg31,savg32,savg33,savg34,savg35,savg36,savg37,savg38,savg39,savg40,savg41,savg42,savg43,savg44,savg45,savg46,savg47,savg48,savg49,savg50,savg51,savg52,savg53,savg54,savg55,savg56,savg57,savg58,savg59
0,3911565283,17.354,11.98,10.194,16.704,17.682,23.534,24.018,22.434,20.336,18.93,18.946,29.3,28.138,34.234,18.818,18.772,22.77,17.428,25.982001,17.426,26.538,25.642,23.042,41.167999,16.536,49.926,60.354,14.058,26.298,30.496,14.34,19.976,24.206,15.472,15.792,19.744,21.91,15.718,11.544,15.816,30.906,23.314,15.174,16.274,21.142,17.216,18.132,33.684,21.231999,19.258,20.816,16.148,11.624,33.578,30.16,19.876,13.98,22.292,11.792,13.432


In [26]:
# load models
vote_cols=['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']
models=[f'model{i}' for i in range(len(vote_cols))]
for i in range(len(vote_cols)):
    models[i]=joblib.load(f'{kaggle_temp}/models/{vote_cols[i]}.joblib')


In [27]:
# predicting
train_cols=[col for col in f_test.columns if col not in ['eeg_id']]
p_votes=pd.DataFrame(columns=vote_cols)
for i in range(len(models)):
    p_votes[vote_cols[i]]=models[i].predict(f_test[train_cols])
p_votes

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,0.117647,0.205308,0.328645,0.1,0.070334,0.505382


In [28]:
# generalizing probs
for i in range(len(p_votes)):
   p_votes.loc[i,vote_cols]=p_votes.loc[i,vote_cols].apply(lambda x: (x/sum(p_votes.loc[i,vote_cols])) )
p_votes

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,0.088635,0.154679,0.247601,0.07534,0.052989,0.380755


In [29]:
p_votes=pd.concat([f_test['eeg_id'],p_votes],axis=1)

In [30]:
p_votes

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.088635,0.154679,0.247601,0.07534,0.052989,0.380755


In [31]:
p_votes.to_csv(f'{kaggle_temp}/submission.csv',index=False)